# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17780395780909604921
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357302272
locality {
  bus_id: 1
}
incarnation: 2590683038384495285
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # DONE:Implement
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    tokenized_data = tokenizer.texts_to_sequences(x)
    
    return tokenized_data, tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
print(text_tokenized)
print(max([len(sentence) for sentence in text_tokenized]))
text_padded = pad_sequences(text_tokenized, maxlen= max([len(sentence) for sentence in text_tokenized]), padding='post')
print(text_padded)
print(type(text_padded))

[[1, 2, 4, 5, 6, 7, 1, 8, 9], [10, 11, 12, 2, 13, 14, 15, 16, 3, 17], [18, 19, 3, 20, 21]]
10
[[ 1  2  4  5  6  7  1  8  9  0]
 [10 11 12  2 13 14 15 16  3 17]
 [18 19  3 20 21  0  0  0  0  0]]
<class 'numpy.ndarray'>


In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        length = max([len(sentence) for sentence in x])
        
    text_padded = pad_sequences(x, maxlen= length, padding='post')
    
    return text_padded
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [14]:
from keras.models import Sequential
from keras.layers import SimpleRNN
from keras.callbacks import EarlyStopping, ModelCheckpoint

def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # DONE: Build the layers
    model = Sequential()
    model.add(GRU(units= 128, input_shape = input_shape[1:], return_sequences=True))
    model.add(Dense(french_vocab_size, activation="softmax"))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=0.001),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_gru_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)
simple_gru_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=30, validation_split=0.2,
                    callbacks=[EarlyStopping(monitor='val_acc', patience=4),
                                                  ModelCheckpoint('models/Simple_GRU_best.hdf5', monitor='val_acc',
                                                                  verbose = 1,save_best_only = True)])

# Print prediction(s)
print(logits_to_text(simple_gru_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 16s 143us/step - loss: 2.0902 - acc: 0.5360 - val_loss: 1.5381 - val_acc: 0.5944
Epoch 2/30
110288/110288 [==============================] - 15s 140us/step - loss: 1.4126 - acc: 0.6219 - val_loss: 1.3059 - val_acc: 0.6384
Epoch 3/30
110288/110288 [==============================] - 15s 140us/step - loss: 1.2364 - acc: 0.6497 - val_loss: 1.1759 - val_acc: 0.6545
Epoch 4/30
110288/110288 [==============================] - 16s 141us/step - loss: 1.1424 - acc: 0.6619 - val_loss: 1.1077 - val_acc: 0.6652
Epoch 5/30
110288/110288 [==============================] - 16s 141us/step - loss: 1.0874 - acc: 0.6695 - val_loss: 1.0662 - val_acc: 0.6671
Epoch 6/30
110288/110288 [==============================] - 16s 141us/step - loss: 1.0479 - acc: 0.6744 - val_loss: 1.0292 - val_acc: 0.6758
Epoch 7/30
110288/110288 [==============================] - 15s 140us/step - loss: 1.0163 - acc: 0.6793

Epoch 27/30
110288/110288 [==============================] - 15s 141us/step - loss: 0.6604 - acc: 0.7987 - val_loss: 0.6598 - val_acc: 0.8001
Epoch 28/30
110288/110288 [==============================] - 16s 141us/step - loss: 0.6528 - acc: 0.8009 - val_loss: 0.6550 - val_acc: 0.8022
Epoch 29/30
110288/110288 [==============================] - 16s 141us/step - loss: 0.6466 - acc: 0.8023 - val_loss: 0.6464 - val_acc: 0.7999
Epoch 30/30
110288/110288 [==============================] - 16s 141us/step - loss: 0.6389 - acc: 0.8045 - val_loss: 0.6430 - val_acc: 0.8007
new jersey est parfois chaud en mois de il est est il en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [15]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # DONE: Implement
    model = Sequential()
    model.add(Embedding(input_dim = english_vocab_size, output_dim = output_sequence_length, 
                        input_length =output_sequence_length))
    model.add(GRU(units= 128, return_sequences=True))
    model.add(Dense(french_vocab_size, activation="softmax"))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=0.001),
                  metrics=['accuracy'])
    return model
tests.test_embed_model(embed_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train the neural network
embedded_model = embed_model(tmp_x.shape,max_french_sequence_length,
                             english_vocab_size,french_vocab_size+1)

embedded_model.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=40, validation_split=0.2,
                   callbacks=[EarlyStopping(monitor='val_acc', patience=2),
                              ModelCheckpoint('models/Embed_model_best.hdf5',
                                              monitor='val_acc', verbose = 1,save_best_only = True)])

# Print prediction(s)
print(logits_to_text(embedded_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/40
110288/110288 [==============================] - 12s 107us/step - loss: 3.1245 - acc: 0.4268 - val_loss: 2.4205 - val_acc: 0.4918
Epoch 2/40
110288/110288 [==============================] - 11s 102us/step - loss: 1.9590 - acc: 0.5523 - val_loss: 1.5634 - val_acc: 0.6098
Epoch 3/40
110288/110288 [==============================] - 11s 101us/step - loss: 1.3837 - acc: 0.6472 - val_loss: 1.2155 - val_acc: 0.6933
Epoch 4/40
110288/110288 [==============================] - 11s 102us/step - loss: 1.0804 - acc: 0.7332 - val_loss: 0.9469 - val_acc: 0.7622
Epoch 5/40
110288/110288 [==============================] - 11s 102us/step - loss: 0.8452 - acc: 0.7847 - val_loss: 0.7482 - val_acc: 0.8061
Epoch 6/40
110288/110288 [==============================] - 11s 102us/step - loss: 0.6803 - acc: 0.8198 - val_loss: 0.6181 - val_acc: 0.8329
Epoch 7/40
110288/110288 [==============================] - 11s 102us/step - loss: 0.5726 - acc: 0.8425

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [18]:
from keras.layers import Bidirectional

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # DONE: Implement
    model = Sequential()
    model.add(Bidirectional(GRU(units= 128, return_sequences=True), input_shape = input_shape[1:]))
    model.add(Dense(french_vocab_size, activation="softmax"))
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=0.001),
                  metrics=['accuracy'])
    return model
tests.test_bd_model(bd_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bidirec_model = bd_model(tmp_x.shape,
                         max_french_sequence_length,
                         english_vocab_size,
                         french_vocab_size+1)
bidirec_model.fit(tmp_x, preproc_french_sentences, batch_size=256, epochs=30, validation_split=0.2,
                  callbacks=[EarlyStopping(monitor='val_acc', patience=4),
                                                  ModelCheckpoint('models/Bidirec_GRU_best.hdf5', monitor='val_acc',
                                                                  verbose = 1,save_best_only = True)])

# Print prediction(s)
print(logits_to_text(bidirec_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/30
110288/110288 [==============================] - 27s 242us/step - loss: 1.7299 - acc: 0.5918 - val_loss: 1.2663 - val_acc: 0.6444
Epoch 2/30
110288/110288 [==============================] - 26s 238us/step - loss: 1.1690 - acc: 0.6636 - val_loss: 1.0902 - val_acc: 0.6785
Epoch 3/30
110288/110288 [==============================] - 26s 237us/step - loss: 1.0384 - acc: 0.6857 - val_loss: 0.9948 - val_acc: 0.6894
Epoch 4/30
110288/110288 [==============================] - 26s 239us/step - loss: 0.9633 - acc: 0.6969 - val_loss: 0.9311 - val_acc: 0.7012
Epoch 5/30
110288/110288 [==============================] - 26s 239us/step - loss: 0.9077 - acc: 0.7064 - val_loss: 0.8847 - val_acc: 0.7089
Epoch 6/30
110288/110288 [==============================] - 26s 239us/step - loss: 0.8654 - acc: 0.7133 - val_loss: 0.8464 - val_acc: 0.7180
Epoch 7/30
110288/110288 [==============================] - 26s 237us/step - loss: 0.8310 - acc: 0.7201

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [12]:
from keras.models import Sequential
from keras.layers import SimpleRNN
from keras.callbacks import EarlyStopping, ModelCheckpoint

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    model = Sequential()
    model.add(GRU(128,input_shape =input_shape[1:], return_sequences=False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences=True))
    model.add(Dense(french_vocab_size, activation='softmax'))
    
    model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
    return model

tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
encdeced_model = encdec_model(tmp_x.shape,
                         max_french_sequence_length,
                         english_vocab_size,
                         french_vocab_size+1)
encdeced_model.fit(tmp_x, preproc_french_sentences, batch_size=375, epochs=40, validation_split=0.2,
                    callbacks=[EarlyStopping(monitor='val_acc', patience=3),
                                                  ModelCheckpoint('models/EncDec_GRU_best.hdf5', monitor='val_acc',
                                                                  verbose = 1,save_best_only = True)])

# Print prediction(s)
print(logits_to_text(encdeced_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/40
110288/110288 [==============================] - 23s 211us/step - loss: 2.6459 - acc: 0.4723 - val_loss: 2.1047 - val_acc: 0.5005
Epoch 2/40
110288/110288 [==============================] - 21s 194us/step - loss: 1.9091 - acc: 0.5350 - val_loss: 1.8545 - val_acc: 0.5332
Epoch 3/40
110288/110288 [==============================] - 21s 192us/step - loss: 1.6719 - acc: 0.5663 - val_loss: 1.6031 - val_acc: 0.5761
Epoch 4/40
110288/110288 [==============================] - 21s 191us/step - loss: 1.5187 - acc: 0.5946 - val_loss: 1.4693 - val_acc: 0.6030
Epoch 5/40
110288/110288 [==============================] - 21s 192us/step - loss: 1.4089 - acc: 0.6157 - val_loss: 1.3846 - val_acc: 0.6185
Epoch 6/40
110288/110288 [==============================] - 21s 192us/step - loss: 1.3421 - acc: 0.6285 - val_loss: 1.3126 - val_acc: 0.6334
Epoch 7/40
110288/110288 [==============================] - 21s 192us/step - loss: 1.2961 - acc: 0.6346

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [13]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # DONE: Implement
    model = Sequential()
    model.add(Embedding(input_dim = english_vocab_size, output_dim = output_sequence_length, 
                        input_length =input_shape[-1]))
    model.add(Bidirectional(GRU(units= 128, return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(units= 64, return_sequences=True)))
    model.add(Dense(french_vocab_size, activation='softmax'))
    
    model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(lr=0.002),
              metrics=['accuracy'])
    return model

tests.test_model_final(model_final)


print('Final Model Loaded')
# TODO: Train the final model

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
#tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1)) 

# Train the neural network
final_model = model_final(tmp_x.shape,
                         max_french_sequence_length,
                         english_vocab_size,
                         french_vocab_size+1)
final_model.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=50, validation_split=0.2,
                    callbacks=[EarlyStopping(monitor='val_acc', patience=4),
                                                  ModelCheckpoint('models/Final_GRU_best.hdf5', monitor='val_loss',
                                                                  verbose = 1,save_best_only = True)])

# Print prediction(s)
print(logits_to_text(final_model.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded
Train on 110288 samples, validate on 27573 samples
Epoch 1/50
110288/110288 [==============================] - 35s 319us/step - loss: 2.5637 - acc: 0.4737 - val_loss: 1.8517 - val_acc: 0.5458
Epoch 2/50
110288/110288 [==============================] - 32s 294us/step - loss: 1.6151 - acc: 0.5815 - val_loss: 1.4625 - val_acc: 0.6110
Epoch 3/50
110288/110288 [==============================] - 32s 294us/step - loss: 1.3237 - acc: 0.6353 - val_loss: 1.2470 - val_acc: 0.6601
Epoch 4/50
110288/110288 [==============================] - 32s 293us/step - loss: 1.1303 - acc: 0.6856 - val_loss: 1.0450 - val_acc: 0.7084
Epoch 5/50
110288/110288 [==============================] - 32s 293us/step - loss: 0.9929 - acc: 0.7186 - val_loss: 0.9562 - val_acc: 0.7256
Epoch 6/50
110288/110288 [==============================] - 32s 294us/step - loss: 0.8931 - acc: 0.7410 - val_loss: 0.8450 - val_acc: 0.7526
Epoch 7/50
110288/110288 [==============================] - 32s 292us/step - loss: 0

Epoch 27/50
110288/110288 [==============================] - 32s 293us/step - loss: 0.1158 - acc: 0.9674 - val_loss: 0.1277 - val_acc: 0.9649
Epoch 28/50
110288/110288 [==============================] - 32s 294us/step - loss: 0.1063 - acc: 0.9697 - val_loss: 0.1215 - val_acc: 0.9662
Epoch 29/50
110288/110288 [==============================] - 32s 292us/step - loss: 0.0983 - acc: 0.9720 - val_loss: 0.1100 - val_acc: 0.9690
Epoch 30/50
110288/110288 [==============================] - 32s 290us/step - loss: 0.0943 - acc: 0.9731 - val_loss: 0.1136 - val_acc: 0.9682
Epoch 31/50
110288/110288 [==============================] - 32s 292us/step - loss: 0.1110 - acc: 0.9684 - val_loss: 0.1110 - val_acc: 0.9692
Epoch 32/50
110288/110288 [==============================] - 32s 292us/step - loss: 0.0874 - acc: 0.9749 - val_loss: 0.1056 - val_acc: 0.9709
Epoch 33/50
110288/110288 [==============================] - 32s 294us/step - loss: 0.0794 - acc: 0.9772 - val_loss: 0.1037 - val_acc: 0.9715
Epoch 

## Prediction (IMPLEMENTATION)

In [14]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    tmp_x = pad(x, 15)
    
    final_model_impl = model_final(tmp_x.shape,
                             max_french_sequence_length,
                             english_vocab_size,
                             french_vocab_size+1)
    final_model_impl.fit(tmp_x, preproc_french_sentences, batch_size=512, epochs=50, validation_split=0.2,
                        callbacks=[EarlyStopping(monitor='val_acc', min_delta=0.0075, patience=4),
                                                      ModelCheckpoint('models/Final_impl_GRU_best.hdf5', monitor='val_loss',
                                                                      verbose = 1,save_best_only = True)])
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = final_model_impl.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/50
110288/110288 [==============================] - 30s 269us/step - loss: 2.5425 - acc: 0.4615 - val_loss: 1.9915 - val_acc: 0.5048
Epoch 2/50
110288/110288 [==============================] - 29s 259us/step - loss: 1.7750 - acc: 0.5538 - val_loss: 1.5701 - val_acc: 0.5866
Epoch 3/50
110288/110288 [==============================] - 29s 259us/step - loss: 1.4252 - acc: 0.6143 - val_loss: 1.3053 - val_acc: 0.6427
Epoch 4/50
110288/110288 [==============================] - 29s 262us/step - loss: 1.2179 - acc: 0.6616 - val_loss: 1.1236 - val_acc: 0.6820
Epoch 5/50
110288/110288 [==============================] - 29s 262us/step - loss: 1.0679 - acc: 0.6955 - val_loss: 1.0151 - val_acc: 0.7099
Epoch 6/50
110288/110288 [==============================] - 29s 260us/step - loss: 0.9686 - acc: 0.7184 - val_loss: 0.9857 - val_acc: 0.7092
Epoch 7/50
110288/110288 [==============================] - 29s 261us/step - loss: 0.8897 - acc: 0.7366

Epoch 26/50
110288/110288 [==============================] - 29s 261us/step - loss: 0.1749 - acc: 0.9517 - val_loss: 0.1859 - val_acc: 0.9493
Epoch 27/50
110288/110288 [==============================] - 29s 263us/step - loss: 0.1584 - acc: 0.9564 - val_loss: 0.1733 - val_acc: 0.9530
Epoch 28/50
110288/110288 [==============================] - 29s 261us/step - loss: 0.1486 - acc: 0.9591 - val_loss: 0.1661 - val_acc: 0.9535
Epoch 29/50
110288/110288 [==============================] - 29s 261us/step - loss: 0.1395 - acc: 0.9613 - val_loss: 0.1482 - val_acc: 0.9591
Epoch 30/50
110288/110288 [==============================] - 29s 260us/step - loss: 0.1269 - acc: 0.9649 - val_loss: 0.1444 - val_acc: 0.9596
Epoch 31/50
110288/110288 [==============================] - 29s 260us/step - loss: 0.1269 - acc: 0.9642 - val_loss: 0.1382 - val_acc: 0.9616
Epoch 32/50
110288/110288 [==============================] - 29s 260us/step - loss: 0.1233 - acc: 0.9653 - val_loss: 0.1472 - val_acc: 0.9591
Epoch 

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

In [15]:
!!python -m nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 412675 bytes to machine_translation.html',
 '[NbConvertApp] Converting notebook machine_translation_UC.ipynb to html',
 '[NbConvertApp] Writing 306470 bytes to machine_translation_UC.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?